# 03 Model Training

### Imports

In [1]:
import os
import gc
import pickle
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from sys import getsizeof
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.optimizers import SGD
from keras import layers
from keras import regularizers

### Paths

In [2]:
BASE_PATH = './data'
ORIGINAL_IMAGES = f'{BASE_PATH}/unsplit'
IMAGE_PICKLES = f'{BASE_PATH}/pickles'
IMAGE_NPYS = f'{BASE_PATH}/npys'

In [3]:
CATEGORIES = os.listdir(ORIGINAL_IMAGES) # list the names of the categories that in my data
print(str(len(CATEGORIES)),'CATEGORIES are ', CATEGORIES)

8 CATEGORIES are  ['Black_Hooded_Oriole', 'Common_Myna', 'Indian_Peafowl', 'Intermediate_Egret', 'Other', 'Red_Faced_Malkoha', 'Rose_Ringed_Parakeet', 'White_Throated_Kingfisher']


In [4]:
# Image pickle save and load functions

def save_pickle(obj, filename):
    with open(os.path.join(IMAGE_PICKLES, f'{filename}.pkl'), 'wb') as f:  # open a text file
        pickle.dump(obj, f) # serialize the list
    
def load_pickle(filename):
    with open(os.path.join(IMAGE_PICKLES, f'{filename}.pkl'), 'rb') as f:
        return pickle.load(f) # deserialize using load()

In [5]:
X_train = load_pickle('X_train')
y_train = load_pickle('y_train')
X_test = load_pickle('X_test')
y_test = load_pickle('y_test')

In [6]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1900, 320, 400, 3), (1900,), (936, 320, 400, 3), (936,))

In [7]:
y_train

array([5, 5, 6, ..., 0, 1, 1])

In [8]:
y_train.reshape(len(y_train), 1)

array([[5],
       [5],
       [6],
       ...,
       [0],
       [1],
       [1]])

In [9]:
y_train.reshape(len(y_train), 1).shape

(1900, 1)

In [10]:
y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

In [11]:
# convolution layer that we will be used in out model i added batch normalization to accelerate the model and for generalization
def conv_layer(inputs, filters, kernel_size=3, padding="valid"):
    x = layers.Conv2D(filters = filters, kernel_size = kernel_size, padding = padding, 
                      activity_regularizer=regularizers.L2(1e-4), use_bias = False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x

# pooling layer i added dropout cause it help my model to reduce the overfitting
def pooling_layer(inputs, pool_size = 2, dropout_rate=0.0):
    x = layers.MaxPooling2D(pool_size = pool_size)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    return x

# this dense layer i will not only use it for my base model i will use it in the pretrained model too
def dense_layer(inputs, out, dropout_rate = 0.0):
    x = layers.Dense(out, activity_regularizer=regularizers.L2(1e-5))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    return x

In [12]:
keras.backend.clear_session()

In [13]:
inputs = keras.Input(shape = (320, 400, 3))

In [14]:
x = conv_layer(inputs, 64, 3, padding = "same")  # 320 x 400
x = conv_layer(x, 64, 3)                         # 318 x 398
x = pooling_layer(x, 2)                          # 159 x 199

In [15]:
x = conv_layer(x, 64, padding = "same")       # 159 x 199
x = conv_layer(x, 64)                         # 157 x 197
x = pooling_layer(x, 2)                       #  78 x  98

In [16]:
x = conv_layer(x, 64, padding = "same")       # 78 x 98
x = conv_layer(x, 64)                         # 76 x 96
x = pooling_layer(x, 2)                       # 38 x 48

In [17]:
x = layers.Flatten()(x)                       # 38*48*64 = 116,736

In [18]:
x = dense_layer(x, 512, dropout_rate=0.1)

In [19]:
x = layers.Dense(len(CATEGORIES), 
                 activation = "softmax", 
                 activity_regularizer=regularizers.L2(1e-5))(x)
outputs = layers.Dropout(0.1)(x)

In [20]:
base_model = keras.Model(inputs, outputs)

In [21]:
base_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 320, 400, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 320, 400, 64)      1728      
                                                                 
 batch_normalization (Batch  (None, 320, 400, 64)      256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 320, 400, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 318, 398, 64)      36864     
                                                                 
 batch_normalization_1 (Bat  (None, 318, 398, 64)      256       
 chNormalization)                                            

In [22]:
# base_model.compile(optimizer =keras.optimizers.Adam(learning_rate = 0.001),
#                loss = 'categorical_crossentropy',
#                metrics = ['accuracy'])

base_model.compile(optimizer=keras.optimizers.Adam(learning_rate = 0.001),
               loss = 'sparse_categorical_crossentropy',
               metrics = ['accuracy'])

In [ ]:
#fit model
history = base_model.fit(
    X_train, y_train,
    batch_size=100,
    epochs = 10, # adding more epochs will increase the acc like 1% or 2%
    validation_split = 0.2,
    verbose = 1)

Epoch 1/10



In [ ]:
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, "bo", label = "Trianing accuracy")
plt.plot(epochs, val_accuracy, "b-", label = "Validation accuracy")
plt.title("Accuracy on training and validation data")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label = "Trianing loss")
plt.plot(epochs, val_loss, "b-", label = "Validation loss")
plt.title("loss on training and validation data")
plt.title("loss on training and validation data")
plt.legend()
plt.show()

In [ ]:
# testing the model on the testset
# test_model_base = keras.models.load_model("intial_model.keras")
_, test_acc = base_model.evaluate(test_generator)
print(f"The accuracy of the intial model on the test set is : {test_acc:.3f}")